# 司法院透過案號抓取檔案
* 要爬取所有資料

## Import & Install

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import math
import re
import enum

In [3]:
# settings
pd.set_option('display.precision', 0)

In [4]:

labels = ['序號','裁判字號','裁判日期','裁判案由','裁判內容','螢光' ,'補充內容']

In [5]:
# use beautiful soup to parse reviews
# 直接找呈現100則評論的頁面
with open("./受監護宣告.txt") as file:
    urls = file.readlines()
df  = []
for url in urls:
    # Make a GET request to the iframe source URL
    response = requests.get(url, verify=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')
        contents = soup.find('div', class_ = 'int-table col-xs-8')
        tmp = {}

        col_th_elements = soup.find_all('div', class_='col-th')
        col_td_elements = soup.find_all('div', class_='col-td')

        # Iterate over the elements and extract text content
        for col_th, col_td in zip(col_th_elements, col_td_elements):
            col_name = col_th.text.strip()
            col_value = col_td.text.strip()
            tmp[col_name] = col_value

        # content
        content = ''
        contents = soup.find('div', class_ = "jud_content")
        if contents:
            for c in contents.find_all('div'):
                content +=c.text
            tmp['content'] = content
            df.append(tmp)           

    else:
        print("Failed to fetch iframe content:", response.status_code)



In [6]:
tmp_df = pd.DataFrame(df)

In [7]:
tmp_df

,裁判字號：,裁判日期：,裁判案由：,content
0,臺灣臺南地方法院 113 年度監宣字第 286 號民事裁定,民國 113 年 05 月 09 日,監護宣告,臺灣臺南地方法院民事裁定113年度監宣字第286號聲 請 人 施守益 住臺南市南...
1,臺灣臺中地方法院 113 年度監宣字第 370 號民事裁定,民國 113 年 05 月 09 日,許可監護人行為,臺灣臺中地方法院民事裁定113年度監宣字第370號聲 請 人 宋○○ 住○○市○...
2,臺灣高等法院 臺中分院 112 年度重家上字第 31 號民事裁定,民國 113 年 05 月 08 日,分割遺產,臺灣高等法院臺中分院民事裁定112年度重家上字第31號上 訴 人 陳麗花 訴訟代...
3,臺灣高等法院 高雄分院 111 年度醫上字第 9 號民事判決,民國 113 年 05 月 08 日,侵權行為損害賠償,臺灣高等法院高雄分院民事判決111年度醫上字第9號上 訴 人 迦樂醫療財團法人迦樂醫院...
4,臺灣彰化地方法院 112 年度交易字第 807 號刑事判決,民國 113 年 05 月 08 日,過失致重傷害,臺灣彰化地方法院刑事判決112年度交易字第807號公 訴 人 臺灣彰化地方檢察署檢察官...
...,...,...,...,...
495,臺灣臺中地方法院 113 年度監宣字第 82 號民事裁定,民國 113 年 03 月 25 日,監護宣告,臺灣臺中地方法院民事裁定 ...
496,臺灣嘉義地方法院 113 年度輔宣字第 16 號民事裁定,民國 113 年 03 月 25 日,輔助宣告,臺灣嘉義地方法院民事裁定113年度輔宣字第16號聲 請 人 鄭慧芬 相 對 人 ...
497,臺灣士林地方法院 113 年度監宣字第 14 號民事裁定,民國 113 年 03 月 25 日,監護宣告,臺灣士林地方法院民事裁定113年度監宣字第14號聲 請 人 甲〇〇 住○○市○○...
498,臺灣苗栗地方法院 113 年度家聲抗字第 2 號民事裁定,民國 113 年 03 月 25 日,監護宣告,臺灣苗栗地方法院民事裁定113年度家聲抗字第2號抗 告 人 鄭秀珠 住○○市○○...


In [8]:
tmp_df.to_csv('/home/B20711/Law_NER/data/受監護宣告.csv', index=False)